In [2]:
# Bibliotecas padrão
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import requests

import warnings

# Desabilitar todos os warnings
warnings.filterwarnings("ignore")

Comercio = pd.read_csv('Dados/Dados_Embrapa/Comercio/Tbl_comercio.csv', sep=None, engine='python', encoding='utf-8')
Comercio.head()

,id,control,Produto,1970,1971,1972,1973,1974,1975,1976,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,VINHO DE MESA,VINHO DE MESA,98327606,114399031,118377367,116617910,94173324,108031792,139238614,...,206404427,209198468,166769622,176059959,177186273,180446489,215557931,210012238,187939996,187016848
1,2,vm_Tinto,Tinto,83300735,98522869,101167932,98196747,77167303,91528090,116407222,...,178250072,182028785,146646365,154309442,155115499,158519218,189573423,185653678,165067340,165097539
2,3,vm_Rosado,Rosado,107681,542274,7770851,8425617,8891367,7261777,11748047,...,1419855,1409002,1391942,1097426,1972944,1265435,1394901,1931606,2213723,2520748
3,4,vm_Branco,Branco,14919190,15333888,9438584,9995546,8114654,9241925,11083345,...,26734500,25760681,18731315,20653091,20097830,20661836,24589607,22426954,20658933,19398561
4,5,VINHO FINO DE MESA,VINHO FINO DE MESA,4430629,4840369,5602091,7202830,7571802,8848303,14095648,...,20424983,20141631,19630158,15874354,14826143,15684588,24310834,27080445,21533487,18589310


In [13]:
# Manter primeiras 3 colunas e as últimas 15 colunas
comercio_tratada = pd.concat([Comercio.iloc[:, :3], Comercio.iloc[:, -15:]], axis=1)
comercio_tratada.head(10)

# Adiciona a coluna 'categoria' com valores vazios inicialmente
comercio_tratada['categoria'] = None

# Preenche os valores com base na coluna 'control', ignorando NaNs
mask_vm = comercio_tratada['control'].fillna('').str.startswith('vm_')
mask_vv = comercio_tratada['control'].fillna('').str.startswith('vv_')
mask_su = comercio_tratada['control'].fillna('').str.startswith('su_')
mask_de = comercio_tratada['control'].fillna('').str.startswith('de')
mask_fr = comercio_tratada['control'].fillna('').str.startswith('VINHO FR')
mask_or = comercio_tratada['control'].fillna('').str.startswith('VINHO ORG')
mask_ve = comercio_tratada['control'].fillna('').str.startswith('ve')


comercio_tratada.loc[mask_vm, 'categoria'] = 'VINHO DE MESA'
comercio_tratada.loc[mask_vv, 'categoria'] = 'VINHO FINO DE MESA'
comercio_tratada.loc[mask_su, 'categoria'] = 'SUCO'
comercio_tratada.loc[mask_de, 'categoria'] = 'DERIVADOS'
comercio_tratada.loc[mask_fr, 'categoria'] = 'VINHO FRIZANTE'
comercio_tratada.loc[mask_or, 'categoria'] = 'VINHO ORGANICO'
comercio_tratada.loc[mask_ve, 'categoria'] = 'VINHO ESPECIAL'


# Retira os dados somados e cria uma categoria para os dados de produção
colunas_para_manter = ['categoria'] + comercio_tratada.dropna(axis='columns').columns.tolist()
comercio_tratada = comercio_tratada[colunas_para_manter]
comercio_tratada = comercio_tratada.dropna(subset=['categoria'])

In [14]:
comercio_tratada.head(30)

,categoria,id,Produto,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
1,VINHO DE MESA,2,Tinto,193004182,188649074,196562722,173964776,188033494,178250072,182028785,146646365,154309442,155115499,158519218,189573423,185653678,165067340,165097539
2,VINHO DE MESA,3,Rosado,2307580,2036928,1668823,1738134,1777648,1419855,1409002,1391942,1097426,1972944,1265435,1394901,1931606,2213723,2520748
3,VINHO DE MESA,4,Branco,39214217,30556943,32078923,31266661,31779668,26734500,25760681,18731315,20653091,20097830,20661836,24589607,22426954,20658933,19398561
5,VINHO DE MESA,6,Tinto,19576295,15184398,14876896,15443016,19121750,15354938,15572632,15228514,12021684,11150517,11433702,18202453,19337862,15258778,12450606
6,VINHO DE MESA,7,Rosado,213835,236802,211484,150806,214269,164219,169185,172351,182080,262430,503524,993248,1603537,1318396,1214583
7,VINHO DE MESA,8,Branco,13290140,5968959,4878930,6876128,8576915,4905826,4399814,4229293,3670590,3413196,3747362,5115133,6139046,4956314,4924121
8,VINHO FRIZANTE,9,VINHO FRIZANTE,1003596,1851136,1706679,1605767,1764851,1893469,1836167,1727386,1586985,1638337,1826433,2557585,3696762,2875864,2843600
9,VINHO ORGANICO,10,VINHO ORGÂNICO,0,0,0,0,0,0,0,0,0,0,2554,10718,18686,14947,9123
11,VINHO ESPECIAL,12,Tinto,113,293,699,239,2257,7231,593,331,0,0,0,0,0,0,0
12,VINHO ESPECIAL,13,Rosado,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
comercio_final = pd.melt(comercio_tratada, id_vars=['categoria', 'Produto','id'], value_vars= comercio_tratada.iloc[:,-15:], var_name="Ano", value_name="Qtd_Produzida (L)")


In [19]:
comercio_final.head(50)

,categoria,Produto,id,Ano,Qtd Produzida (L)
0,VINHO DE MESA,Tinto,2,2009,193004182
1,VINHO DE MESA,Rosado,3,2009,2307580
2,VINHO DE MESA,Branco,4,2009,39214217
3,VINHO DE MESA,Tinto,6,2009,19576295
4,VINHO DE MESA,Rosado,7,2009,213835
5,VINHO DE MESA,Branco,8,2009,13290140
6,VINHO FRIZANTE,VINHO FRIZANTE,9,2009,1003596
7,VINHO ORGANICO,VINHO ORGÂNICO,10,2009,0
8,VINHO ESPECIAL,Tinto,12,2009,113
9,VINHO ESPECIAL,Rosado,13,2009,0
